In [1]:
import numpy as np
import pandas as pd

from warnings import filterwarnings
filterwarnings(action='ignore', category=FutureWarning)

COKE = '/kaggle/input/coca-cola-stock-live-and-updated/Coca-Cola_stock_history.csv'

df = pd.read_csv(filepath_or_buffer=COKE,)
# we have dates in two formats, with and without time; we need to clean them
df['date'] = df['Date'].str.split().str[0]
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

# now we need to add our trade signal components

df['50d'] = df['Close'].rolling(window=50).mean()
df['200d'] = df['Close'].rolling(window=200).mean()
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,date,year,50d,200d
0,1962-01-02,0.050016,0.051378,0.050016,0.050016,806400,0.0,0,1962-01-02,1962,NaN,NaN
1,1962-01-03,0.049273,0.049273,0.048159,0.048902,1574400,0.0,0,1962-01-03,1962,NaN,NaN
2,1962-01-04,0.049026,0.049645,0.049026,0.049273,844800,0.0,0,1962-01-04,1962,NaN,NaN
3,1962-01-05,0.049273,0.049892,0.048035,0.048159,1420800,0.0,0,1962-01-05,1962,NaN,NaN
4,1962-01-08,0.047787,0.047787,0.046735,0.047664,2035200,0.0,0,1962-01-08,1962,NaN,NaN


In [2]:
from plotly.express import line
for log_y in [False, True]:
    line(data_frame=df, x='date', y='Close', log_y=log_y, ).show()

Let's look at our indicators for just the New Coke era. 

In [3]:
from datetime import datetime

new_df = df[df['date'] > pd.Timestamp('1985-04-23')].copy()

In [4]:
line(data_frame=new_df, x='date', y=['50d', '200d'], log_y=True)

If we buy and hold what is our total return?

In [5]:
new_df['Close'].values[0], new_df['Close'].values[-1], new_df['Close'].values[-1] - new_df['Close'].values[0] 

(0.58621794, 59.38999939, 58.80378145)

In [6]:
print('total return: {:5.2f} pct'.format(100 * new_df['Close'].values[-1]/new_df['Close'].values[0]))
# years
years = (new_df['date'].max() - new_df['date'].min()).days / 365.25
print('years: {}'.format(round(100 * years)/100))

total return: 10131.04 pct
years: 37.51


In [7]:
from math import log
from math import exp

def get_rate(start_value: float, end_value: float, years: float) -> float:
    rate = exp(1.0/years * log(end_value/start_value)) - 1.0
    return rate

annualized_rate = get_rate(start_value=new_df['Close'].values[0], end_value=new_df['Close'].values[-1], years=years)

print('annualized pct rate: {:6.3f}'.format(100 * annualized_rate))

annualized pct rate: 13.103


A 13% annualized return is nothing to sneeze at.

In [8]:
1.13103 ** 37.51

101.34781984451695

In [9]:
profits = []
state = 0
profit = 0
count = 0
for index, row in new_df.iterrows():
    price = row['Close']
    if row['50d']  > row['200d']:
        if state == 0:
            state = 1
            profit -= price
            print('buy: {} price: {:5.2f} profit: {:5.2f}'.format(row['date'].date(), price, profit))
            count += 1
    elif row['50d'] < row['200d']:
        if state == 1:
            state = 0
            profit += price
            print('sell: {} price:  {:5.2f} profit: {:5.2f}'.format(row['date'].date(), price, profit))
            count += 1
    profits.append(profit)
if state == 1:
    state = 0
    count += 1
    profit += price
    print('closeout:  price: {:5.2f} profit: {:5.2f}'.format(price, profit))
print('we made {} trades'.format(count))

buy: 1985-04-24 price:  0.59 profit: -0.59
sell: 1986-11-10 price:   0.99 profit:  0.40
buy: 1986-12-10 price:  1.06 profit: -0.66
sell: 1987-11-12 price:   1.09 profit:  0.43
buy: 1988-08-02 price:  1.07 profit: -0.64
sell: 1992-11-12 price:   5.03 profit:  4.38
buy: 1993-02-16 price:  5.00 profit: -0.62
sell: 1993-04-30 price:   4.86 profit:  4.25
buy: 1993-07-02 price:  5.30 profit: -1.05
sell: 1994-02-23 price:   5.33 profit:  4.28
buy: 1994-08-02 price:  5.57 profit: -1.29
sell: 1997-10-03 price:  16.38 profit: 15.08
buy: 1998-01-16 price: 17.10 profit: -2.02
sell: 1998-09-25 price:  14.88 profit: 12.86
buy: 1999-06-07 price: 18.02 profit: -5.17
sell: 1999-07-14 price:  16.50 profit: 11.34
buy: 2000-01-12 price: 16.49 profit: -5.15
sell: 2000-02-11 price:  14.90 profit:  9.74
buy: 2000-07-19 price: 16.33 profit: -6.59
sell: 2001-03-08 price:  13.96 profit:  7.37
buy: 2001-11-14 price: 13.73 profit: -6.36
sell: 2002-01-31 price:  12.06 profit:  5.70
buy: 2002-03-25 price: 14.33 pro

In [10]:
from plotly.express import scatter
new_df['profit'] = new_df['Close'] + profits
scatter(data_frame=new_df, x='date', y=['Close', 'profit',], log_y=False).show()
scatter(data_frame=new_df, x='date', y=['Close', 'profit',], log_y=True).show()